**Max Subarray**

Given an integer array nums, find the contiguous subarray (containing at least one number) which has the largest sum and return its sum.

Example:
```
Input: [-2,1,-3,4,-1,2,1,-5,4],
Output: 6
Explanation: [4,-1,2,1] has the largest sum = 6.
```
This is an interesting one to think of as a dp program. As we know, it can be done with a single counter instead of a table (in this case an array), so adding that element feels like an unnecessary complication, but it was actually a pretty useful thought exercise. The logic piece is the same: looping from the front of the list, check if the current maxsum + the next number is < the current sum: if so, the past elements are not in the max subarray, so the maxsum should be updated to be the new number. otherwise, the new number can simply be added to maxsum and you can continue. To dp this, instead of keeping a counter you can keep an array of the max sum at each index, and then return the max element in that array. For example:
```
input: [-2,1,-3,4,-1,2,1,-5,4]

maxSum = -inf
sums:  [max(-inf, -2) = -2, max((-2 + 1 = -1), 1) = 1, max(1 + -3 = -2), -3) = -2, max((-2 + 4 = 2), 4) = 4, max((4 + -1 = 3), -1) = 3, max((3 + 2 = 5), 2) = 5, max((5 + 1 = 6), 1) = 6, max((6 + -5 = 1), -5) = 1, max((1 + 4 = 5), 4) = 5]
```
the max element is 6 at index 6, the end of the largest subarray.
However, this unnecessarily adds O(n) space into a solution that works iteratively with O(1) space, so it's not an ideal solution. But it's a great starting point for dp thinking.

In this DP solution, you loop through the list once to get the sums, and then again to get the max of your answers ==> O(n + n --> n). the max taken within the loop is constant time because it is always just comparing 2 numbers. As mentioned, the space is O(n).

In [ ]:
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        maxSum = float("inf") * -1
        n = len(nums)
        ans = [float("inf") * -1] * (n + 1)
        for i in range(n):
            ans[i + 1] = max(ans[i] + nums[i], nums[i])
        return max(ans)

**Unique Paths**

A robot is located at the top-left corner of a m x n grid (marked 'Start' in the diagram below).

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid (marked 'Finish' in the diagram below).

How many possible unique paths are there?

Drawing the answers to a grid really helps solidify the answer here:
```
m = 3, n = 7
paths to each cell in the grid:

1   1   1   1   1   1   1
1   2   3   4   5   6   7
1   3   6  10  15  21  28
```
There is only ever one path to each cell in the top row and leftmost column, because you can only go right or down, respectively, to reach each one. from there, a pattern emerges: each further cell can only be reached by the paths to the cell directly above and cell directly left of it. that means the path to any given cell (after row = 0 and c = 0) is the # of paths to the cell above it + the number of paths to the cell left of it. this trend continues throughout the grid till you reach the final cell. This means that we can keep track of the table above, filled with answers, if we first fill in the givens of the top row and left column, and fill in from the top row and left column from there. After this process, we can simply return the answer in the cell at (m - 1, n - 1) and we're done. 

This process requires us to go through each cell in the grid only once, making our runtime O(m x n). The space required for the table is the same, O(m x n). 

In [ ]:
class Solution:
    def uniquePaths(self, m: int, n: int) -> int:
        grid = []
        for row in range(n):
            grid.append([1] * m)
        for r in range(1, n):
            for c in range(1, m):
                grid[r][c] = grid[r - 1][c] + grid[r][c - 1]
        return grid[n - 1][m - 1]

**Unique Paths II**

A robot is located at the top-left corner of a m x n grid (marked 'Start' in the diagram below).

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid (marked 'Finish' in the diagram below).

Now consider if some obstacles are added to the grids. How many unique paths would there be?

Ok this is trickier for a couple of reasons. First of all when instantiating your answer grid, not only does each obstacle have to be recorded as a 0, but if there's a 0 in the top row or left column, every cell afterward in that row/column is a 0 too, not a 1. If you hit an obstacle, you can't go around it and still only move down and right. Once your grid is instantiated, you also have to make sure not to update obstacle cells in your grid, but instead leave them as 0. Otherwise, however, the logic stays the same. For the number of paths to a cell, you still add the paths to the cell above and to the left of it. obstacles are just 0s in this math. 

This process will objectively take longer than the first unique paths problem, but runtime and space complexity -wise, the answer is the same: O(m x n) for each.

In [ ]:
class Solution:
    def uniquePathsWithObstacles(self, obstacleGrid: List[List[int]]) -> int:
        self.obstacleGrid = obstacleGrid
        self.m = len(obstacleGrid)
        self.n = len(obstacleGrid[0])
        self.grid = self.instantiateGrid()
        return self.countPaths()
        
    
    def instantiateGrid(self):
        grid = []
        for row in range(self.m):
            grid.append([0] * self.n)
        for r in range(self.m):
            if self.obstacleGrid[r][0] == 1:
                break
            grid[r][0] = 1
        for c in range(self.n):
            if self.obstacleGrid[0][c] == 1:
                break
            grid[0][c] = 1
        for r in range(self.m):
            for c in range(self.n):
                if self.obstacleGrid[r][c] == 1:
                    grid[r][c] = 0
        return grid
    
    
    def countPaths(self):
        for r in range(1, self.m):
            for c in range(1, self.n):
                if self.obstacleGrid[r][c] != 1:
                    self.grid[r][c] = self.grid[r - 1][c] + self.grid[r][c - 1]
        return self.grid[self.m - 1][self.n - 1]

**Coin Change**

You are given coins of different denominations and a total amount of money amount. Write a function to compute the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1.

I really don't like this one. So basically, in order to compute the final amount, you need to compute each amount before it. therefore, you should have an array tracking the number of coins needed for each number from 0 cents up to (and including) the given amount, therefore your arrays's size should be `amount + 1` and it should be instantiated with each element as infinity. The known element is that it takes 0 coins to make 0 cents, so we can update our array's first element to be 0. Then, for each coin, we can go from the index at that coin's amount to the end of the array and set that element to min(array[i], array[i - coin] + 1). since we're starting from the front of the array, the first time through this loop for each coin will be the index at that coin's worth, which means you can achieve that amount by using that coin once. array[i - coin] when i == coin is index 0, which we set to the value 0. so array[coin] will be set to 1 for each coin. From there, for the first coin, it will build on that and each index at which index % coin == 0 will be updated as array[index - coin] + 1. then it will do the same thing with the next coin, but now many more [index - coin]s will have been filled by the previous coin, and thus be accessible for adding this new coin to. so for example, say your coins are [1, 2, 5] and your amount is 11. 
```
first loop, coin = 1
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
second loop, coin = 2
[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5,  6]
third loop, coin = 5
[0, 1, 1, 2, 2, 1, 2, 2, 3, 3, 2,  3]
```
the final element in the list is 3, which is correct as 5 + 5 + 1 = 11.
now say the amount was still 11 but the coins were [2, 5].
```
first loop, coin = 2
[0, inf, 1, inf, 2, inf, 3, inf, 4, inf, 5, inf]
second loop, coin = 5
[0, inf, 1, inf, 2, 1,   3, inf, 4, inf, 2, inf]
```
the last element is inf, so we can return -1 because it's impossible.

You loop len(coins), let's call that k, * amount + 1, let's call that n + 1. so the runtime of this algorithm is O(n x k). the space of the answers array is n + 1 so space complexity is O(n).

In [ ]:
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        table = [float("inf")] * (amount + 1)
        table[0] = 0
        for coin in coins:
            for x in range(coin, amount + 1):
                table[x] = min(table[x], table[x - coin] + 1)
        if table[amount] == float('inf'): return -1
        return table[amount]

**Jump Game II**

Given an array of non-negative integers, you are initially positioned at the first index of the array.

Each element in the array represents your maximum jump length at that position.

Your goal is to reach the last index in the minimum number of jumps. Assume you can always make it to the end. 

this is so hard!!!!!

OK for each index a from 0 to len(array), f(a) = minimum number of jumps to get there. ex:
```
input:    [2,3,1,1,4]
minjumps: [0,1,1,2,2]
```
because it takes 0 jumps to be at index 0, then you can move 2 steps (index 1 and 2) with a single jump of 2, from there you need to figure out which of the two options that that jump hits (3 at index 1 and 1 at index 2) will take you farther. in this case that's the 3. make sure not to overwrite the 1s already filled in by the previous jump by just skipping over those indices, and filling in the rest with 2s. in this case, 2 jumps gets you exactly to the end of the array, but in the case that it goes over, you have to make sure that you're not going over the length of the array.

yikes ok so time and space complexity. you are actually only hitting each index once because once you've looked at an index in the inner for loop, it is skipped in the outer while loop with `i += currLength`. so the runtime is O(n). and the space is just the array of size n, so also O(n). hurray!

In [ ]:
class Solution:
    def jump(self, nums: List[int]) -> int:
        n = len(nums)
        jumps = [float("inf")] * n
        jumps[0] = 0
        currLength = nums[0]
        i = 1
        while i < n:
            maxLenLeft = 0
            for j in range(currLength):
                if (i + j) >= n:
                    break
                jumps[i + j] = jumps[i - 1] + 1
                lenLeft = nums[i + j] - (currLength - j)
                if lenLeft > maxLenLeft:
                    maxLenLeft = lenLeft
            i += currLength
            currLength = maxLenLeft + 1
        return jumps[n - 1]